In [2]:
import pandas as pd 
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
print(df_train.shape, df_test.shape)

(131662, 14) (87395, 13)


In [6]:
df_train.columns

Index(['Trip_ID', 'Trip_Distance', 'Type_of_Cab', 'Customer_Since_Months',
       'Life_Style_Index', 'Confidence_Life_Style_Index', 'Destination_Type',
       'Customer_Rating', 'Cancellation_Last_1Month', 'Var1', 'Var2', 'Var3',
       'Gender', 'Surge_Pricing_Type'],
      dtype='object')

In [10]:
df_train.Trip_ID.nunique(dropna=True)
## This proves that we have all uniuqe Trip_ID's

131662

In [12]:
for col_name in df_train.columns :
    print("Number of unique entries for " , col_name , " are ",df_train[col_name].nunique(dropna=True))

Number of unique entries for  Trip_ID  are  131662
Number of unique entries for  Trip_Distance  are  10326
Number of unique entries for  Type_of_Cab  are  5
Number of unique entries for  Customer_Since_Months  are  11
Number of unique entries for  Life_Style_Index  are  55978
Number of unique entries for  Confidence_Life_Style_Index  are  3
Number of unique entries for  Destination_Type  are  14
Number of unique entries for  Customer_Rating  are  3931
Number of unique entries for  Cancellation_Last_1Month  are  9
Number of unique entries for  Var1  are  122
Number of unique entries for  Var2  are  58
Number of unique entries for  Var3  are  96
Number of unique entries for  Gender  are  2
Number of unique entries for  Surge_Pricing_Type  are  3


In [17]:
df_train[['Var1','Var2','Var3','Type_of_Cab','Surge_Pricing_Type']].corr(method ='spearman')

,Var1,Var2,Var3,Surge_Pricing_Type
Var1,1.000000,-0.055995,-0.068324,-0.042341
Var2,-0.055995,1.000000,0.673239,0.003800
Var3,-0.068324,0.673239,1.000000,-0.044620
Surge_Pricing_Type,-0.042341,0.003800,-0.044620,1.000000


In [29]:
df_train.fillna(-1,inplace=True)

In [46]:
y = df_train['Surge_Pricing_Type']
## X = df_train.drop('Surge_Pricing_Type', axis=1)
X = df_train[['Cancellation_Last_1Month','Customer_Rating','Life_Style_Index','Customer_Since_Months','Trip_Distance','Var1','Var2','Var3']]


In [47]:
from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=1000, 
                               bootstrap = True,
                               max_features = 'sqrt')

print('Fitting the model')
# Fit on training data
model.fit(X, y)
print('Done with modelling')

Fitting the model


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [50]:
df_test = df_test[['Cancellation_Last_1Month','Customer_Rating','Life_Style_Index','Customer_Since_Months','Trip_Distance','Var1','Var2','Var3']]
df_test.fillna(-1,inplace=True)
# Actual class predictions
rf_predictions = model.predict(df_test)
# Probabilities for each class
rf_probs = model.predict_proba(df_test)[:, 1]
print(rf_predictions[1:10])

NameError: name 'rf_prediction' is not defined

In [55]:
print(rf_predictions[1:10],rf_probs[1:10])

[2 1 3 3 2 3 2 3 3] [0.588 0.326 0.377 0.411 0.53  0.329 0.419 0.403 0.339]


In [57]:
df_submit = pd.read_csv('sample_submission.csv')
df_submit['Surge_Pricing_Type'] = rf_predictions
df_submit.to_csv('Naive_Sumbit.csv',index=False , header=True)

In [58]:
importances = model.feature_importances_
importances

array([0.03387768, 0.19200712, 0.16735104, 0.08579779, 0.20630627,
       0.07603564, 0.10864489, 0.12997957])